# Downstream analysis
In this notebook the downstream analysis is shown. The most important fuctions are illustrate and the features are extracted for all the cells in the WSI.

In [1]:
# Imports
from shapely.geometry import LineString
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import os
import func_utils as fu
import sys

In [2]:
# Import the json
path='./Output/InstanceSegmentation/' #Input forlder
sample='' #Specific slide name with the extation
gson=os.path.join(path,sample)

In [3]:
#import the json file with all the cell (shapely geometry)
cellDoc = pd.read_json(gson)
cellDoc["geometry"] = cellDoc["geometry"].apply(shapely.geometry.shape)
cells = gpd.GeoDataFrame(cellDoc).set_geometry('geometry')
cells

,type,id,geometry,properties
0,Feature,PathCellDetection,"POLYGON ((34929.012 45173.004, 34929.012 45177...","{'isLocked': 'false', 'measurements': [], 'cla..."
1,Feature,PathCellDetection,"POLYGON ((34465.012 45248.995, 34466.005 45249...","{'isLocked': 'false', 'measurements': [], 'cla..."
2,Feature,PathCellDetection,"POLYGON ((34735.012 45234.003, 34735.012 45242...","{'isLocked': 'false', 'measurements': [], 'cla..."
3,Feature,PathCellDetection,"POLYGON ((34908.012 45246.995, 34915.826 45254...","{'isLocked': 'false', 'measurements': [], 'cla..."
4,Feature,PathCellDetection,"POLYGON ((34714.012 45246.005, 34714.012 45251...","{'isLocked': 'false', 'measurements': [], 'cla..."
...,...,...,...,...
48978,Feature,PathCellDetection,"POLYGON ((56825.012 49970.005, 56825.012 49972...","{'isLocked': 'false', 'measurements': [], 'cla..."
48979,Feature,PathCellDetection,"POLYGON ((56595.012 49979.005, 56595.012 49987...","{'isLocked': 'false', 'measurements': [], 'cla..."
48980,Feature,PathCellDetection,"POLYGON ((56716.012 49983.003, 56716.012 49987...","{'isLocked': 'false', 'measurements': [], 'cla..."
48981,Feature,PathCellDetection,"POLYGON ((56615.011 49998.989, 56630.989 49998...","{'isLocked': 'false', 'measurements': [], 'cla..."


In [4]:
# Function to extract information regarding the numbers of Neoplastic, Inflammatory and Connective cells present in the WSI slide
data,cells_list,poly_list=fu.cell_counting(cells)
print('Neoplastic cells: %i\nInflammatory cells: %i\nConnective cells: %i' %(data['neoplastic'],data['inflammatory'],data['connective']))

Neoplastic cells: 11021
Inflammatory cells: 7067
Connective cells: 30874


In [5]:
# Function to extract the percentage of Neoplastic, Inflammatory and Connective cells present in the WSI slide
norm_data=fu.N_F_C_norm(data)
print('Neoplastic percentage: %.3f\nInflammatory percentage: %.3f\nConnective percentage: %.3f' %(norm_data['neoplastic'],norm_data['inflammatory'],norm_data['connective']))

Neoplastic percentage: 0.225
Inflammatory percentage: 0.144
Connective percentage: 0.631


In [6]:
# Function to extract the Shannon Index
ShannonIndex=fu.Shannon_indx(data)        
print('Shannon Index: %.3f'% ShannonIndex)

Shannon Index: 0.393


In [7]:
# Morphological features extraction from cells
#Area
areas=fu.geoArea(cells_list)
#Solidity
solidit=fu.geoSolidity(cells_list)
#Eccentricity 
eccentr=fu.geoEccentricity(cells_list)
#Rotation
rot_ang=fu.geoRotation(cells_list)
#Minor Axis, Major Axis and Aspect Ratio
minor_axis, major_axis, aspect_ratio=fu.geoAxis(cells_list)
#Roundness
roundness=fu.geoRoundness(cells_list)
#Perimeter
perimeter=fu.geoPerimeter(cells_list)


In [8]:
# Function to obtain all the features in one variable
cell_feat=fu.cell_features(cells)
cell_feat.head()

,CellType,Area,Solidity,Eccentricity,Rotation,MinorAxis,MajorAxis,Aspect ratio,Roundness,Perimeter
0,connective,463.533808,1.000000,0.771605,0.785398,19.775103,31.087962,1.572076,0.610671,80.028605
1,connective,192.028756,1.000000,0.934263,-1.249056,9.462207,26.535741,2.804392,0.347227,59.694483
2,neoplastic,558.555043,0.989933,0.842325,1.231501,19.778324,36.696525,1.855391,0.528112,93.717395
3,connective,383.224720,0.993968,0.870330,-1.325603,16.228003,32.952393,2.030588,0.449355,81.140904
4,neoplastic,282.824283,1.000000,0.320356,1.570786,17.976246,18.976351,1.055635,1.000003,60.527147
